In [15]:
# Libraries for processing

import numpy as np
import pandas as pd
import datetime as dt
import quandl
import matplotlib.pyplot as plt
import seaborn as sns
import traces

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [16]:
# ease of display
pd.set_option('display.max_columns',None)

## Import External Excel Data:

In [17]:
# Make list of labels of all Excel files to be read and concatenated
# Only years from 2010 through end of 2015 are being considered.
# Timeline of Quantitative Easing policy.

file_list_treasuries = []

for i in range(2010,2015):
    for j in range(1,5):
        file_list_treasuries.append(f'tsy_data_{i}_q{j}.xls')
    
file_list_treasuries

['tsy_data_2010_q1.xls',
 'tsy_data_2010_q2.xls',
 'tsy_data_2010_q3.xls',
 'tsy_data_2010_q4.xls',
 'tsy_data_2011_q1.xls',
 'tsy_data_2011_q2.xls',
 'tsy_data_2011_q3.xls',
 'tsy_data_2011_q4.xls',
 'tsy_data_2012_q1.xls',
 'tsy_data_2012_q2.xls',
 'tsy_data_2012_q3.xls',
 'tsy_data_2012_q4.xls',
 'tsy_data_2013_q1.xls',
 'tsy_data_2013_q2.xls',
 'tsy_data_2013_q3.xls',
 'tsy_data_2013_q4.xls',
 'tsy_data_2014_q1.xls',
 'tsy_data_2014_q2.xls',
 'tsy_data_2014_q3.xls',
 'tsy_data_2014_q4.xls']

In [19]:
# First 2 quarters of 2010 are not available.
file_list_treasuries = file_list_treasuries[2:]

In [20]:
# Concatenate primary DataFrame from Excel files.
df = pd.concat([pd.read_excel(f'../data/Treasuries/{file}') for file in file_list_treasuries])

In [21]:
df.head()

,U.S. TREASURY SECURITIES DATA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,"JULY 22 TO SEPTEMBER 30, 2010",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trade date,Settlement date,Transaction category,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty
3,2010-08-17 00:00:00,2010-08-18 00:00:00,Purchase,25,TSY,TN 02.375 02/28/15,912828MR8 ...,104.773,275900,26.4693,Banc of America Securities LLC
4,2010-08-17 00:00:00,2010-08-18 00:00:00,Purchase,50,TSY,TN 02.375 10/31/14,912828LS7 ...,104.922,354959,52.8159,Banc of America Securities LLC


## Pre-Processing:

In [22]:
# Apply column names
labels = list(df.iloc[2])
df.columns = labels

In [23]:
df.head()

,Trade date,Settlement date,Transaction category,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty
0,"JULY 22 TO SEPTEMBER 30, 2010",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trade date,Settlement date,Transaction category,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty
3,2010-08-17 00:00:00,2010-08-18 00:00:00,Purchase,25,TSY,TN 02.375 02/28/15,912828MR8 ...,104.773,275900,26.4693,Banc of America Securities LLC
4,2010-08-17 00:00:00,2010-08-18 00:00:00,Purchase,50,TSY,TN 02.375 10/31/14,912828LS7 ...,104.922,354959,52.8159,Banc of America Securities LLC


In [24]:
# Check for nulls
df.isnull().sum()

Trade date                                     18
Settlement date                                36
Transaction category                           36
Trade amount (in millions, USD)                36
Issuer                                         36
Security description                           36
CUSIP                                          36
Price                                          36
Accrued interest (in dollars, USD)             36
Total amount transferred (in millions, USD)    36
Counterparty                                   36
dtype: int64

In [25]:
# Too few nulls for data quality to be damaged by dropping.
df.dropna(inplace=True)

In [26]:
df.shape

(80926, 11)

In [27]:
df.dtypes

Trade date                                     object
Settlement date                                object
Transaction category                           object
Trade amount (in millions, USD)                object
Issuer                                         object
Security description                           object
CUSIP                                          object
Price                                          object
Accrued interest (in dollars, USD)             object
Total amount transferred (in millions, USD)    object
Counterparty                                   object
dtype: object

In [28]:
# Finding and removing white space in string categories.
# Finding and removing errors of label name being in cell were discovered as well.
# check for name mismatch . Removes other rows with offending columns as well

# Example:
# This is when the string 'Trade date' was found instead of an actual date value

df[df['Trade date'] == 'Trade date'].shape

(18, 11)

In [29]:
df = df[df['Trade date'] != 'Trade date'].copy()

In [30]:
# extra whitespace
df['Transaction category'].value_counts()

Purchase    71745
Sale         7120
Sale         2043
Name: Transaction category, dtype: int64

In [31]:
# Many corporate names for effectively same entity
df['Counterparty'].value_counts()

Morgan Stanley & Co. LLC                                                   18259
BNP Paribas Securities Corp.                                               12077
Goldman, Sachs & Co.                                                        7585
Barclays Capital Inc.                                                       6820
Merrill Lynch, Pierce, Fenner & Smith Incorporated                          4154
Citigroup Global Markets Inc.                                               3135
Credit Suisse Securities (USA) LLC                                          2832
Deutsche Bank Securities Inc.                                               2794
RBS Securities Inc.                                                         2510
Nomura Securities International, Inc.                                       2304
BMO Capital Markets Corp.                                                   1793
RBC Capital Markets, LLC                                                    1567
HSBC Securities (USA) Inc.  

In [32]:
# Change data types ot integers and floats

df['Trade amount (in millions, USD)'] = df['Trade amount (in millions, USD)'].astype(int)
df['Price'] = df['Price'].astype(float)
df['Accrued interest (in dollars, USD)'] = df['Accrued interest (in dollars, USD)'].astype(float)
df['Total amount transferred (in millions, USD)'] = df['Total amount transferred (in millions, USD)'].astype(float)

In [34]:
# remover for whitespace
def space_remover(string):
    return string.replace(' ','')

In [35]:
df['Transaction category'] = df['Transaction category'].map(space_remover)
df['Transaction category'].value_counts()

Purchase    71745
Sale         9163
Name: Transaction category, dtype: int64

In [36]:
df['Security description'] = df['Security description'].map(space_remover)

In [37]:
# Condense singular names of Counterparty business entity

def CounterPartyNamer(string):
    if 'Goldman' in string:
        string = 'Goldman, Sachs & Co.'
        return string
    elif 'J.' in string:
        string = 'J.P. Morgan Securities LLC'
        return string
    elif 'Stanley' in string:
        string = 'Morgan Stanley & Co. LLC'
        return string
    elif 'RBC' in string:
        string = 'RBC Capital Markets, LLC'
        return string
    elif 'Barclay' in string:
        string = 'Barclays Capital Inc.'
        return string
    elif 'Jeffer' in string:
        string = 'Jefferies & Company, Inc.'
        return string
    elif 'Cantor' in string:
        string = 'Cantor Fitzgerald & Co.'
        return string
    elif 'Nomura' in string:
        string = 'Nomura Securities International, Inc.'
        return string
    elif 'RBS' in string:
        string = 'RBS Securities Inc.'
        return string
    elif 'Daiwa' in string:
        string = 'Daiwa Capital Markets America Inc.'
        return string
    elif 'HSBC' in string:
        string = 'HSBC Securities (USA) Inc.'
        return string
    elif 'Citigroup' in string:
        string = 'Citigroup Global Markets Inc.'
        return string
    elif 'BNP' in string:
        string = 'BNP Paribas Securities Corp.'
        return string
    elif 'UBS' in string:
        string = 'UBS Securities LLC'
        return string
    elif 'BMO' in string:
        string = 'BMO Capital Markets Corp.'
        return string
    elif 'Societe' in string:
        string = 'SG Americas Securities, LLC'
        return string
    elif 'Suisse' in string or 'SUISSE' in string:
        string = 'Credit Suisse Securities (USA) LLC'
        return string
    elif 'Deutsche' in string:
        string = 'Deutsche Bank Securities Inc.'
        return string
    elif 'Scotia' in string:
        string = 'Bank of Nova Scotia, New York Agency'
        return string
    elif 'Mizuho' in string:
        string = 'Mizuho Securities USA Inc.'
        return string
    elif 'TD' in string:
        string = 'TD Securities (USA) LLC'
        return string
    else:
        return string

In [38]:
# Counterparty names have now been standardized

df['Counterparty'] = df['Counterparty'].map(CounterPartyNamer)
df['Counterparty'].value_counts()

Morgan Stanley & Co. LLC                              19076
BNP Paribas Securities Corp.                          12110
Goldman, Sachs & Co.                                   7725
Barclays Capital Inc.                                  6857
Merrill Lynch, Pierce, Fenner & Smith Incorporated     4154
Citigroup Global Markets Inc.                          3185
Credit Suisse Securities (USA) LLC                     2886
Deutsche Bank Securities Inc.                          2829
RBS Securities Inc.                                    2535
J.P. Morgan Securities LLC                             2358
Nomura Securities International, Inc.                  2321
UBS Securities LLC                                     1797
BMO Capital Markets Corp.                              1793
RBC Capital Markets, LLC                               1578
HSBC Securities (USA) Inc.                             1577
Mizuho Securities USA Inc.                             1464
Jefferies & Company, Inc.               

## Feature Engineerging From Security Description

In [39]:
# Functions for partitioning Interest rate, type, and maturity date from Security Description.

def TermFinder(string):
    return string[-8:]

def CouponFinder(string):
    return string[-14:-8]

def SecType(string):
    return string[:2]

In [40]:
df['Type'] = df['Security description'].map(SecType)
df['Type'].value_counts()

TN    40900
TB    36409
II     3599
Name: Type, dtype: int64

In [41]:
df['Coupon'] = df['Security description'].map(CouponFinder)
df['Coupon']

3      02.375
4      02.375
5      02.375
6      02.625
7      02.375
        ...  
466    03.000
467    03.125
468    02.750
469    04.500
470    03.875
Name: Coupon, Length: 80908, dtype: object

In [42]:
# additional extra character remover from string.

def BFinder(string):
    if 'B' in string:
        string = '0'
        return string
    else:
        return string

In [43]:
df['Coupon'].value_counts()

04.375    6479
03.125    5481
04.250    3920
02.750    3735
04.500    3527
          ... 
05.375      93
09.875      91
04.875      57
05.125      41
09.250      34
Name: Coupon, Length: 68, dtype: int64

In [44]:
df['Term'] = df['Security description'].map(TermFinder)
df['Term']

3      02/28/15
4      10/31/14
5      08/31/14
6      12/31/14
7      09/30/14
         ...   
466    05/15/42
467    02/15/43
468    08/15/42
469    02/15/36
470    08/15/40
Name: Term, Length: 80908, dtype: object

In [45]:
# Standardizing correct names for type of Treasury
type_dict = {'TN':'T-Note','TB':'T-Bill','B1':'Bond','II':'Inf_Indexed'}

In [46]:
df['Type'] = df['Type'].map(type_dict)
df['Type'].value_counts()

T-Note         40900
T-Bill         36409
Inf_Indexed     3599
Name: Type, dtype: int64

In [47]:
df['Coupon'] = df['Coupon'].astype(float)

In [48]:
df['Term'] = pd.to_datetime(df['Term'])
df['Trade date'] = pd.to_datetime(df['Trade date'])
df['Settlement date'] = pd.to_datetime(df['Settlement date'])

In [49]:
# def TermFinder(string):
#     return string[-10:]

# def CouponFinder(string):
#     return string[-15:-8]

# def SecType(string):
#     return string[:3]

In [50]:
# def DateFixer(string):
#     if ' ' in string:
#         string = string.strip()
#         string = string[-8:]
#         return string
#     else:
#         return string

In [51]:
df.dtypes

Trade date                                     datetime64[ns]
Settlement date                                datetime64[ns]
Transaction category                                   object
Trade amount (in millions, USD)                         int64
Issuer                                                 object
Security description                                   object
CUSIP                                                  object
Price                                                 float64
Accrued interest (in dollars, USD)                    float64
Total amount transferred (in millions, USD)           float64
Counterparty                                           object
Type                                                   object
Coupon                                                float64
Term                                           datetime64[ns]
dtype: object

In [52]:
df.head()

,Trade date,Settlement date,Transaction category,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty,Type,Coupon,Term
3,2010-08-17,2010-08-18,Purchase,25,TSY,TN02.37502/28/15,912828MR8 ...,104.773438,275900.14,26.469260,Banc of America Securities LLC,T-Note,2.375,2015-02-28
4,2010-08-17,2010-08-18,Purchase,50,TSY,TN02.37510/31/14,912828LS7 ...,104.921875,354959.24,52.815897,Banc of America Securities LLC,T-Note,2.375,2014-10-31
5,2010-08-17,2010-08-18,Purchase,10,TSY,TN02.37508/31/14,912828LK4 ...,104.968750,110360.05,10.607235,Banc of America Securities LLC,T-Note,2.375,2014-08-31
6,2010-08-17,2010-08-18,Purchase,50,TSY,TN02.62512/31/14,912828ME7 ...,105.937500,174762.23,53.143512,Banc of America Securities LLC,T-Note,2.625,2014-12-31
7,2010-08-17,2010-08-18,Purchase,100,TSY,TN02.37509/30/14,912828LQ1 ...,104.945312,908469.94,105.853782,Barclays Capital Inc.,T-Note,2.375,2014-09-30


## Quandl: Retrieve Time Indexed Yield Curve information 

In [53]:
quandl.ApiConfig.api_key = '-szaxxRjfjXjAhBgEzYR'

In [54]:
# API call for U.S. Treasury Yield Curve data.
yc = quandl.get("USTREASURY/YIELD", authtoken="-szaxxRjfjXjAhBgEzYR")

In [55]:
yc.shape

(7738, 12)

In [56]:
yc = yc.loc['2010':'2014'].copy()

In [57]:
yc.shape

(1251, 12)

In [58]:
df['Trade date']

3     2010-08-17
4     2010-08-17
5     2010-08-17
6     2010-08-17
7     2010-08-17
         ...    
466   2014-10-27
467   2014-10-27
468   2014-10-27
469   2014-10-27
470   2014-10-27
Name: Trade date, Length: 80908, dtype: datetime64[ns]

In [59]:
# https://stackoverflow.com/questions/39019591/duplicated-rows-when-merging-dataframes-in-python
# Join yield curve data on Treasury Transaction data by date.
# Many duplicates as transactions are intraday and Yield Curve interest rates are daily.

yc['Trade date'] = yc.index
df = pd.merge(df,yc.drop_duplicates(),on=['Trade date'])

In [60]:
df.head()

,Trade date,Settlement date,Transaction category,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty,Type,Coupon,Term,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2010-08-17,2010-08-18,Purchase,25,TSY,TN02.37502/28/15,912828MR8 ...,104.773438,275900.14,26.469260,Banc of America Securities LLC,T-Note,2.375,2015-02-28,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77
1,2010-08-17,2010-08-18,Purchase,50,TSY,TN02.37510/31/14,912828LS7 ...,104.921875,354959.24,52.815897,Banc of America Securities LLC,T-Note,2.375,2014-10-31,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77
2,2010-08-17,2010-08-18,Purchase,10,TSY,TN02.37508/31/14,912828LK4 ...,104.968750,110360.05,10.607235,Banc of America Securities LLC,T-Note,2.375,2014-08-31,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77
3,2010-08-17,2010-08-18,Purchase,50,TSY,TN02.62512/31/14,912828ME7 ...,105.937500,174762.23,53.143512,Banc of America Securities LLC,T-Note,2.625,2014-12-31,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77
4,2010-08-17,2010-08-18,Purchase,100,TSY,TN02.37509/30/14,912828LQ1 ...,104.945312,908469.94,105.853782,Barclays Capital Inc.,T-Note,2.375,2014-09-30,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77


In [61]:
# https://stackoverflow.com/questions/40923820/pandas-timedelta-in-months
# Feature Engineering Remaining time to maturity when treasury was purchased

df['Time to Maturity (Years)'] = (df['Term'] - df['Trade date']) / np.timedelta64(1,'Y')
df['Time to Maturity (Months)'] = (df['Term'] - df['Trade date']) / np.timedelta64(1,'M')

In [62]:
# Renaming type of Treasury.

def BondDef(string):
    if 'Bill' in string:
        string = 'T-Bond'
        return string
    elif string == 'Bond':
        string = 'T-Bill'
        return string
    else:
        return string

In [63]:
df['Type'] = df['Type'].map(BondDef)

### Export file for use in EDA

In [64]:
df.reset_index(inplace=True)

In [66]:
df.shape

(80908, 29)

In [67]:
# df.to_csv('../data/csv_exports/edaFINAL.csv')

### Encoding and Processing for Models:

In [355]:
# First is purchase category
df = pd.get_dummies(df,columns=['Transaction category'],drop_first=True)

In [356]:
# First is inflation indexed instrument
df = pd.get_dummies(df,columns=['Type'],drop_first=True)

In [357]:
df.dtypes

index                                                   int64
Trade date                                     datetime64[ns]
Settlement date                                datetime64[ns]
Trade amount (in millions, USD)                         int64
Issuer                                                 object
Security description                                   object
CUSIP                                                  object
Price                                                 float64
Accrued interest (in dollars, USD)                    float64
Total amount transferred (in millions, USD)           float64
Counterparty                                           object
Coupon                                                float64
Term                                           datetime64[ns]
1 MO                                                  float64
2 MO                                                  float64
3 MO                                                  float64
6 MO    

In [358]:
df.head()

,index,Trade date,Settlement date,"Trade amount (in millions, USD)",Issuer,Security description,CUSIP,Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Counterparty,Coupon,Term,1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR,Time to Maturity (Years),Time to Maturity (Months),Transaction category_Sale,Type_T-Bond,Type_T-Note
0,0,2010-08-17,2010-08-18,25,TSY,TN02.37502/28/15,912828MR8 ...,104.773438,275900.14,26.469260,Banc of America Securities LLC,2.375,2015-02-28,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.533974,54.407688,0,0,1
1,1,2010-08-17,2010-08-18,50,TSY,TN02.37510/31/14,912828LS7 ...,104.921875,354959.24,52.815897,Banc of America Securities LLC,2.375,2014-10-31,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.205425,50.465102,0,0,1
2,2,2010-08-17,2010-08-18,10,TSY,TN02.37508/31/14,912828LK4 ...,104.968750,110360.05,10.607235,Banc of America Securities LLC,2.375,2014-08-31,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.038413,48.460954,0,0,1
3,3,2010-08-17,2010-08-18,50,TSY,TN02.62512/31/14,912828ME7 ...,105.937500,174762.23,53.143512,Banc of America Securities LLC,2.625,2014-12-31,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.372437,52.469250,0,0,1
4,4,2010-08-17,2010-08-18,100,TSY,TN02.37509/30/14,912828LQ1 ...,104.945312,908469.94,105.853782,Barclays Capital Inc.,2.375,2014-09-30,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.120550,49.446601,0,0,1


In [359]:
# 2 Month rate feature will be dropped as it is entirely null.
df.isnull().sum()

index                                              0
Trade date                                         0
Settlement date                                    0
Trade amount (in millions, USD)                    0
Issuer                                             0
Security description                               0
CUSIP                                              0
Price                                              0
Accrued interest (in dollars, USD)                 0
Total amount transferred (in millions, USD)        0
Counterparty                                       0
Coupon                                             0
Term                                               0
1 MO                                               0
2 MO                                           80908
3 MO                                               0
6 MO                                               0
1 YR                                               0
2 YR                                          

In [360]:
# Droping other features that are not relevant for modeling.
df.drop(columns=['CUSIP','Security description','index','Issuer','2 MO'],inplace=True)

In [361]:
df.dtypes

Trade date                                     datetime64[ns]
Settlement date                                datetime64[ns]
Trade amount (in millions, USD)                         int64
Price                                                 float64
Accrued interest (in dollars, USD)                    float64
Total amount transferred (in millions, USD)           float64
Counterparty                                           object
Coupon                                                float64
Term                                           datetime64[ns]
1 MO                                                  float64
3 MO                                                  float64
6 MO                                                  float64
1 YR                                                  float64
2 YR                                                  float64
3 YR                                                  float64
5 YR                                                  float64
7 YR    

In [362]:
# Separate data for modeling encoded counterparty
# Two DataFrames: One with encoded counterparties, another without.

counterparty_df = pd.get_dummies(df,columns=['Counterparty'],drop_first=True)

In [363]:
# condense data to each day with aggregates for option
# Aggregating useful measures on individual day
# https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns
# This DataFrame was ultimately not used

dailydata = df.groupby('Trade date').agg({'Trade amount (in millions, USD)':['sum','mean'],
                                          'Price':'mean',
                                          'Counterparty':'count',
                                          'Coupon':'mean',
                                          'Transaction category_Sale':'count',
                                          'Time to Maturity (Years)':'mean',
                                          'Time to Maturity (Months)':'mean',
                                          'Total amount transferred (in millions, USD)':['sum','mean']
                                         })

In [364]:
daily_df = pd.merge(dailydata,yc.drop_duplicates(),on=['Trade date'])

/Users/ash/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [365]:
daily_df.head()

,Trade date,"(Trade amount (in millions, USD), sum)","(Trade amount (in millions, USD), mean)","(Price, mean)","(Counterparty, count)","(Coupon, mean)","(Transaction category_Sale, count)","(Time to Maturity (Years), mean)","(Time to Maturity (Months), mean)","(Total amount transferred (in millions, USD), sum)","(Total amount transferred (in millions, USD), mean)",1 MO,2 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
0,2010-08-17,2551,33.129870,105.687043,77,2.564935,77,4.388687,52.664246,2711.817050,35.218403,0.16,NaN,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77
1,2010-08-19,3609,73.653061,107.213249,49,3.071429,49,6.550247,78.602963,3903.637809,79.666078,0.16,NaN,0.16,0.19,0.25,0.49,0.74,1.41,2.02,2.58,3.37,3.66
2,2010-08-24,1350,50.000000,101.663194,27,1.259259,27,2.813554,33.762652,1373.310600,50.863356,0.17,NaN,0.16,0.20,0.27,0.50,0.72,1.36,1.95,2.50,3.28,3.57
3,2010-08-26,1415,37.236842,141.454770,38,6.963816,38,15.814295,189.771540,2085.439888,54.879997,0.17,NaN,0.16,0.19,0.25,0.51,0.77,1.38,1.97,2.50,3.25,3.53
4,2010-08-30,360,21.176471,113.124081,17,2.139706,17,11.773161,141.277934,421.420783,24.789458,0.16,NaN,0.14,0.19,0.26,0.50,0.75,1.39,1.98,2.54,3.30,3.60


In [366]:
daily_df.shape

(822, 23)

In [367]:
df.drop(columns='Counterparty',inplace=True)

### Interpolating Evenly Spaced Time Intervals that Match Row Number: Traces

In [368]:
df.index = df['Trade date']

In [369]:
# Iniitalize traces object for resampling uneven time index
# traces was imported above.

ts = traces.TimeSeries(df['Trade date'])

In [370]:
# https://stackoverflow.com/questions/25234941/python-regularise-irregular-time-series-with-linear-interpolation/39730384
# Trail and error was used to find number of minutes that keep same number of rows

ts1 = ts.sample(
    sampling_period=dt.timedelta(minutes=27.2845),
    start=dt.datetime(2010,8,17),
    end=dt.datetime(2014,10,28),
)

In [371]:
# rows are now evenly spaced.
pd.DataFrame(ts1).head()

,0,1
0,2010-08-17 00:00:00.000,Trade date 2010-08-17 2010-08-17 2010-08-17 ...
1,2010-08-17 00:27:17.070,Trade date 2010-08-17 2010-08-17 2010-08-17 ...
2,2010-08-17 00:54:34.140,Trade date 2010-08-17 2010-08-17 2010-08-17 ...
3,2010-08-17 01:21:51.210,Trade date 2010-08-17 2010-08-17 2010-08-17 ...
4,2010-08-17 01:49:08.280,Trade date 2010-08-17 2010-08-17 2010-08-17 ...


In [372]:
# Make evenly spaceed time series rows as index.
eventime = pd.DataFrame(ts1)
df.index = eventime[eventime.columns[0]]

In [373]:
df.head()

,Trade date,Settlement date,"Trade amount (in millions, USD)",Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Coupon,Term,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR,Time to Maturity (Years),Time to Maturity (Months),Transaction category_Sale,Type_T-Bond,Type_T-Note
0,,,,,,,,,,,,,,,,,,,,,,,,
2010-08-17 00:00:00.000,2010-08-17,2010-08-18,25,104.773438,275900.14,26.469260,2.375,2015-02-28,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.533974,54.407688,0,0,1
2010-08-17 00:27:17.070,2010-08-17,2010-08-18,50,104.921875,354959.24,52.815897,2.375,2014-10-31,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.205425,50.465102,0,0,1
2010-08-17 00:54:34.140,2010-08-17,2010-08-18,10,104.968750,110360.05,10.607235,2.375,2014-08-31,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.038413,48.460954,0,0,1
2010-08-17 01:21:51.210,2010-08-17,2010-08-18,50,105.937500,174762.23,53.143512,2.625,2014-12-31,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.372437,52.469250,0,0,1
2010-08-17 01:49:08.280,2010-08-17,2010-08-18,100,104.945312,908469.94,105.853782,2.375,2014-09-30,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.120550,49.446601,0,0,1


In [374]:
df.shape

(80908, 24)

In [375]:
# Same index procedure for DataFrame with Encoded Counterparties
counterparty_df.index = eventime[eventime.columns[0]]

## Final Exports for Modeling:

In [376]:
df.to_csv('../data/csv_exports/nocp_FINAL_modeling.csv')
counterparty_df.to_csv('../data/csv_exports/FINAL_modeling.csv')

In [377]:
# df = pd.read_csv('../data/csv_exports/nocp_FINAL_modeling.csv')
# counterparty_df = pd.read_csv('../data/csv_exports/FINAL_modeling.csv')

In [378]:
# verify results
df

,Trade date,Settlement date,"Trade amount (in millions, USD)",Price,"Accrued interest (in dollars, USD)","Total amount transferred (in millions, USD)",Coupon,Term,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR,Time to Maturity (Years),Time to Maturity (Months),Transaction category_Sale,Type_T-Bond,Type_T-Note
0,,,,,,,,,,,,,,,,,,,,,,,,
2010-08-17 00:00:00.000,2010-08-17,2010-08-18,25,104.773438,275900.14,26.469260,2.375,2015-02-28,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.533974,54.407688,0,0,1
2010-08-17 00:27:17.070,2010-08-17,2010-08-18,50,104.921875,354959.24,52.815897,2.375,2014-10-31,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.205425,50.465102,0,0,1
2010-08-17 00:54:34.140,2010-08-17,2010-08-18,10,104.968750,110360.05,10.607235,2.375,2014-08-31,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.038413,48.460954,0,0,1
2010-08-17 01:21:51.210,2010-08-17,2010-08-18,50,105.937500,174762.23,53.143512,2.625,2014-12-31,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.372437,52.469250,0,0,1
2010-08-17 01:49:08.280,2010-08-17,2010-08-18,100,104.945312,908469.94,105.853782,2.375,2014-09-30,0.16,0.17,0.19,0.26,0.52,0.78,1.44,2.06,2.64,3.47,3.77,4.120550,49.446601,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-10-27 21:57:54.210,2014-10-27,2014-10-28,35,99.890625,473641.30,35.435360,3.000,2042-05-15,0.03,0.02,0.06,0.11,0.41,0.81,1.51,1.94,2.27,2.75,3.04,27.548820,330.585844,0,1,0
2014-10-27 22:25:11.280,2014-10-27,2014-10-28,25,101.953125,157099.18,25.645380,3.125,2043-02-15,0.03,0.02,0.06,0.11,0.41,0.81,1.51,1.94,2.27,2.75,3.04,28.304483,339.653792,0,1,0
2014-10-27 22:52:28.350,2014-10-27,2014-10-28,35,94.875000,193546.20,33.399796,2.750,2042-08-15,0.03,0.02,0.06,0.11,0.41,0.81,1.51,1.94,2.27,2.75,3.04,27.800708,333.608493,0,1,0
